In [41]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import webdriver_manager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException, WebDriverException
import pandas as pd
from connection import db_string
import sqlalchemy as sql
from sqlalchemy.orm import sessionmaker
import csv

# Webscraping

### Collecting list of links for all 'Modern' events in time range

In [5]:
#setting up web driver and linking it to a tournament result search on MTGGoldfish.com
#the search is of all recorded tournaments in the "Modern" format for the year of 2022
driver = webdriver.Firefox()
driver.get('https://www.mtggoldfish.com/tournament_searches/create?utf8=%E2%9C%93&tournament_search%5Bname%5D=&tournament_search%5Bformat%5D=modern&tournament_search%5Bdate_range%5D=01%2F01%2F2022+-+12%2F31%2F2022&commit=Search')
driver.implicitly_wait(5)

In [6]:
# for loop for to go through each event on a page

#closing ad that gets in way of click
try:
    ad = driver.find_element(By.TAG_NAME, 'polygon')
    ad.click()
except StaleElementReferenceException:
    pass
except NoSuchElementException: 
    pass
except ElementNotInteractableException:
    pass

#getting page number to loop through
pagination = driver.find_element(By.CLASS_NAME, 'pagination')
num_pages = int(pagination.find_element(By.XPATH, "//li[@class='page-item'][4]").text)

#list for event links
links = []

for i in range(num_pages-1):
    #closing ad that gets in way of click
    try:
        ad = driver.find_element(By.TAG_NAME, 'polygon')
        ad.click()
    except StaleElementReferenceException:
        pass
    except NoSuchElementException: 
        pass
    except ElementNotInteractableException:
        pass
    
    #navigate to list of tournament events
    results = driver.find_element(By.TAG_NAME, "tbody")

    #get all events on page as list
    events = results.find_elements(By.TAG_NAME, 'tr')

    # loop through the links and save the links in a list
    for event in events:
        link = event.find_element(By.TAG_NAME, 'a').get_attribute('href')
        links.append(link)
    print(f"page {i+1}")
    driver.find_element(By.LINK_TEXT, '→').click()

print(links)

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10
page 11
page 12
page 13
page 14
page 15
page 16
page 17
page 18
page 19
page 20
page 21
page 22
page 23
page 24
['https://www.mtggoldfish.com/tournament/41887', 'https://www.mtggoldfish.com/tournament/41942', 'https://www.mtggoldfish.com/tournament/41893', 'https://www.mtggoldfish.com/tournament/41891', 'https://www.mtggoldfish.com/tournament/41900', 'https://www.mtggoldfish.com/tournament/41899', 'https://www.mtggoldfish.com/tournament/41898', 'https://www.mtggoldfish.com/tournament/41902', 'https://www.mtggoldfish.com/tournament/41946', 'https://www.mtggoldfish.com/tournament/41908', 'https://www.mtggoldfish.com/tournament/41907', 'https://www.mtggoldfish.com/tournament/41913', 'https://www.mtggoldfish.com/tournament/41912', 'https://www.mtggoldfish.com/tournament/41918', 'https://www.mtggoldfish.com/tournament/41856', 'https://www.mtggoldfish.com/tournament/41925', 'https://www.mtggoldfish.com/tournament/41951', 

In [7]:
#now can begin looping through list of links to get decks

#for loop of links navigates to that event page
    #for loop to collect list of links for deck lists

#new for loop that goes through list of decklist links
    #decklist scraping for loop


### Collecting list of url links to every deck in each tournament

In [8]:
#creating variable for deck links
deck_links = []

#loop to go through all tournaments
for link in links:
    
    #loads tournament page
    driver.get(f'{link}')
    
    #saving deck elements as list
    try:
        table = driver.find_element(By.CLASS_NAME, 'table-tournament')
    except NoSuchElementException:
        pass
    try:
        decks = table.find_elements(By.XPATH, "//a[starts-with(@href, '/deck/') and not(contains(@href, 'visual')) and not(contains(@href, 'custom'))]")
        
        for deck in decks:
            deck_links.append(deck.get_attribute('href'))
        
    except StaleElementReferenceException:
        pass
    print('event done')



event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done
event done

In [9]:
#saving links to csv as a backup
deck_links_df = pd.DataFrame(deck_links)
deck_links_df.to_csv('decklinks.csv')

In [10]:
length = len(deck_links)
length

18441

In [12]:
cards_df = pd.DataFrame(columns=('Card Name', 'Quantity', 'Date'))
driver =webdriver.Firefox()
save_state = []

#initializing variables to keep progress
length = len(deck_links)
i = 1

for link in deck_links:
    
    #waits until page is correctly loaded. if it errors out the exception handling will reload the page
    try:
        driver.get(f'{link}')
        
        element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//nav")))
        
    except WebDriverException:
        driver.implicitly_wait(5)
        driver = webdriver.Firefox()
        driver.get(f'{link}')
        driver.implicitly_wait(5)
        
    #closing ad that gets in way of click
    try:
        ad = driver.find_element(By.TAG_NAME, 'polygon')
        ad.click()
    except StaleElementReferenceException:
        pass
    except NoSuchElementException: 
        pass
    except ElementNotInteractableException:
        pass

    #collecting the info of the deck and event
    
    info = driver.find_element(By.CLASS_NAME, "deck-container-information")
    #split to find Date: and then get the three items after that entry 
    
    string = info.find_element(By.XPATH, "//*[text()[contains(.,'Deck Date')]]").text
    strings = string.split()
    start = strings.index('Date:')
    
    #save those three as the date
    date_string = strings[start+1:start+4]
    date = date_string[0] +' '+ date_string[1] +" "+ date_string[2]
    print(date)
    
    #get a list of the cards 
    card_list = driver.find_element(By.CLASS_NAME, 'deck-view-deck-table')
    cards = card_list.find_elements(By.TAG_NAME, 'tr')
    
    for card in cards:

        if card.get_attribute('class') == 'deck-category-header':
            continue

          #added a try clause here. Failed originally because a card was entered wrong on the tournament website
            #so there was no hyperlink to the card
        try:
            quant = card.find_element(By.CLASS_NAME, 'text-right').text

            name = card.find_element(By.CLASS_NAME, 'card_id').text

            data = {"Card Name": name, "Quantity": int(quant), "Date":date}

            card_data = pd.DataFrame(data, columns=("Card Name", "Quantity", "Date"), index=[0])

            cards_df= pd.concat([cards_df, card_data], ignore_index=True)
        except NoSuchElementException:
            pass

    #scraping of deck done
    #display progress
    print(f"Deck {i} of {length}")
    
    #saving checkpoints
    if i % 50 == 0:
        cards_df.to_csv('cards_df.csv')
        save_state=[]
        
        for link in deck_links[i+1:]:
            save_state.append(link)
            
        save_state_df = pd.DataFrame(save_state)
        save_state_df.to_csv('save_state.csv', index=False)
        print('Checkpoint saved')
        
    i+=1


Dec 31, 2022
Deck 1 of 18441
Dec 31, 2022
Deck 2 of 18441
Dec 31, 2022
Deck 3 of 18441
Dec 31, 2022
Deck 4 of 18441
Dec 31, 2022
Deck 5 of 18441
Dec 31, 2022
Deck 6 of 18441
Dec 31, 2022
Deck 7 of 18441
Dec 31, 2022
Deck 8 of 18441
Dec 31, 2022
Deck 9 of 18441
Dec 31, 2022
Deck 10 of 18441
Dec 31, 2022
Deck 11 of 18441
Dec 31, 2022
Deck 12 of 18441
Dec 31, 2022
Deck 13 of 18441
Dec 31, 2022
Deck 14 of 18441
Dec 31, 2022
Deck 15 of 18441
Dec 31, 2022
Deck 16 of 18441
Dec 31, 2022
Deck 17 of 18441
Dec 31, 2022
Deck 18 of 18441
Dec 31, 2022
Deck 19 of 18441
Dec 31, 2022
Deck 20 of 18441
Dec 31, 2022
Deck 21 of 18441
Dec 31, 2022
Deck 22 of 18441
Dec 31, 2022
Deck 23 of 18441
Dec 31, 2022
Deck 24 of 18441
Dec 31, 2022
Deck 25 of 18441
Dec 31, 2022
Deck 26 of 18441
Dec 31, 2022
Deck 27 of 18441
Dec 31, 2022
Deck 28 of 18441
Dec 31, 2022
Deck 29 of 18441
Dec 31, 2022
Deck 30 of 18441
Dec 31, 2022
Deck 31 of 18441
Dec 31, 2022
Deck 32 of 18441
Dec 30, 2022
Deck 33 of 18441
Dec 30, 2022
Deck 3

Deck 266 of 18441
Dec 25, 2022
Deck 267 of 18441
Dec 25, 2022
Deck 268 of 18441
Dec 25, 2022
Deck 269 of 18441
Dec 25, 2022
Deck 270 of 18441
Dec 25, 2022
Deck 271 of 18441
Dec 25, 2022
Deck 272 of 18441
Dec 25, 2022
Deck 273 of 18441
Dec 25, 2022
Deck 274 of 18441
Dec 25, 2022
Deck 275 of 18441
Dec 25, 2022
Deck 276 of 18441
Dec 25, 2022
Deck 277 of 18441
Dec 25, 2022
Deck 278 of 18441
Dec 25, 2022
Deck 279 of 18441
Dec 25, 2022
Deck 280 of 18441
Dec 25, 2022
Deck 281 of 18441
Dec 25, 2022
Deck 282 of 18441
Dec 25, 2022
Deck 283 of 18441
Dec 25, 2022
Deck 284 of 18441
Dec 24, 2022
Deck 285 of 18441
Dec 24, 2022
Deck 286 of 18441
Dec 24, 2022
Deck 287 of 18441
Dec 24, 2022
Deck 288 of 18441
Dec 24, 2022
Deck 289 of 18441
Dec 24, 2022
Deck 290 of 18441
Dec 24, 2022
Deck 291 of 18441
Dec 24, 2022
Deck 292 of 18441
Dec 24, 2022
Deck 293 of 18441
Dec 24, 2022
Deck 294 of 18441
Dec 24, 2022
Deck 295 of 18441
Dec 24, 2022
Deck 296 of 18441
Dec 24, 2022
Deck 297 of 18441
Dec 24, 2022
Deck 298

Dec 20, 2022
Deck 528 of 18441
Dec 20, 2022
Deck 529 of 18441
Dec 20, 2022
Deck 530 of 18441
Dec 20, 2022
Deck 531 of 18441
Dec 20, 2022
Deck 532 of 18441
Dec 20, 2022
Deck 533 of 18441
Dec 20, 2022
Deck 534 of 18441
Dec 20, 2022
Deck 535 of 18441
Dec 20, 2022
Deck 536 of 18441
Dec 20, 2022
Deck 537 of 18441
Dec 20, 2022
Deck 538 of 18441
Dec 20, 2022
Deck 539 of 18441
Dec 20, 2022
Deck 540 of 18441
Dec 20, 2022
Deck 541 of 18441
Dec 20, 2022
Deck 542 of 18441
Dec 20, 2022
Deck 543 of 18441
Dec 20, 2022
Deck 544 of 18441
Dec 20, 2022
Deck 545 of 18441
Dec 20, 2022
Deck 546 of 18441
Dec 20, 2022
Deck 547 of 18441
Dec 20, 2022
Deck 548 of 18441
Dec 20, 2022
Deck 549 of 18441
Dec 20, 2022
Deck 550 of 18441
Checkpoint saved
Dec 20, 2022
Deck 551 of 18441
Dec 19, 2022
Deck 552 of 18441
Dec 19, 2022
Deck 553 of 18441
Dec 19, 2022
Deck 554 of 18441
Dec 19, 2022
Deck 555 of 18441
Dec 19, 2022
Deck 556 of 18441
Dec 19, 2022
Deck 557 of 18441
Dec 19, 2022
Deck 558 of 18441
Dec 19, 2022
Deck 559 

NoSuchElementException: Message: Unable to locate element: .card_id


In [13]:
cards_df.to_csv('cards_df.csv')

In [ ]:
### loading in list of links from save state

In [52]:
cards_df

,Unnamed: 0,Index,Card Name,Quantity,Date
0,0.0,NaN,Card Name,Quantity,Date
1,1.0,0.0,Ornithopter,4,31-Dec-22
2,2.0,1.0,Esper Sentinel,4,31-Dec-22
3,3.0,2.0,Gingerbrute,1,31-Dec-22
4,4.0,3.0,Giver of Runes,2,31-Dec-22
...,...,...,...,...,...
81295,NaN,NaN,Chromatic Star,4,"Sep 17, 2022"
81296,NaN,NaN,Expedition Map,4,"Sep 17, 2022"
81297,NaN,NaN,Relic of Progenitus,2,"Sep 17, 2022"
81298,NaN,NaN,Oblivion Stone,2,"Sep 17, 2022"


In [35]:
csv_file = pd.read_csv('save_state.csv', header= None, names = ['Index', 'Link'])
save_state_links = csv_file['Index'].tolist()

save_state_links.remove('0')


In [36]:
#crashed at xx8 so database has entries from those 8 decks already
# so start 8in on the save state
save_state_links = save_state_links[95:]
len(save_state_links)

10216

In [33]:
### loading in card dataframe from save state
cards_df = pd.read_csv('cards_df.csv')
#cards_df=cards_df.drop(cards_df.columns[[0,1,2,3]], axis=1)
cards_df

,Card Name,Quantity,Date
0,Card Name,Quantity,Date
1,Ornithopter,4,31-Dec-22
2,Esper Sentinel,4,31-Dec-22
3,Gingerbrute,1,31-Dec-22
4,Giver of Runes,2,31-Dec-22
...,...,...,...
123048,Voice of the Blessed,4,"Jul 31, 2022"
123049,"Heliod, Sun-Crowned",4,"Jul 31, 2022"
123050,"Ajani, Strength of the Pride",4,"Jul 31, 2022"
123051,Proclamation of Rebirth,3,"Jul 31, 2022"


In [37]:
save_state_links

['https://www.mtggoldfish.com/deck/4992568',
 'https://www.mtggoldfish.com/deck/4992570',
 'https://www.mtggoldfish.com/deck/4992573',
 'https://www.mtggoldfish.com/deck/4992567',
 'https://www.mtggoldfish.com/deck/4992569',
 'https://www.mtggoldfish.com/deck/4992571',
 'https://www.mtggoldfish.com/deck/4992575',
 'https://www.mtggoldfish.com/deck/4992576',
 'https://www.mtggoldfish.com/deck/4992577',
 'https://www.mtggoldfish.com/deck/4992578',
 'https://www.mtggoldfish.com/deck/4992579',
 'https://www.mtggoldfish.com/deck/4992580',
 'https://www.mtggoldfish.com/deck/4992581',
 'https://www.mtggoldfish.com/deck/4992582',
 'https://www.mtggoldfish.com/deck/4992583',
 'https://www.mtggoldfish.com/deck/4992584',
 'https://www.mtggoldfish.com/deck/4992585',
 'https://www.mtggoldfish.com/deck/4992586',
 'https://www.mtggoldfish.com/deck/4992587',
 'https://www.mtggoldfish.com/deck/4992588',
 'https://www.mtggoldfish.com/deck/4992589',
 'https://www.mtggoldfish.com/deck/4992590',
 'https://

In [42]:
driver =webdriver.Firefox()
save_state = []

#initializing variables to keep progress
length = len(save_state_links)
i = 1



for link in save_state_links:
    
    try:
        driver.get(f'{link}')
        
        element = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, "//nav")))
        
    except WebDriverException:
        driver.implicitly_wait(5)
        driver = webdriver.Firefox()
        driver.get(f'{link}')
        driver.implicitly_wait(5)
    #closing ad that gets in way of click
    try:
        ad = driver.find_element(By.TAG_NAME, 'polygon')
        ad.click()
    except StaleElementReferenceException:
        pass
    except NoSuchElementException: 
        pass
    except ElementNotInteractableException:
        pass

    #collecting the info of the deck and event
    
    info = driver.find_element(By.CLASS_NAME, "deck-container-information")
    #split to find Date: and then get the three items after that entry 
    
    string = info.find_element(By.XPATH, "//*[text()[contains(.,'Deck Date')]]").text
    strings = string.split()
    start = strings.index('Date:')
    
    #save those three as the date
    date_string = strings[start+1:start+4]
    date = date_string[0] +' '+ date_string[1] +" "+ date_string[2]
    print(date)
    
    #get a list of the cards 
    card_list = driver.find_element(By.CLASS_NAME, 'deck-view-deck-table')
    cards = card_list.find_elements(By.TAG_NAME, 'tr')
    
    for card in cards:

        if card.get_attribute('class') == 'deck-category-header':
            continue
            
            #added a try clause here. Failed originally because a card was entered wrong on the tournament website
            #so there was no hyperlink to the card
        try:
            quant = card.find_element(By.CLASS_NAME, 'text-right').text

            name = card.find_element(By.CLASS_NAME, 'card_id').text

            data = {"Card Name": name, "Quantity": int(quant), "Date":date}

            card_data = pd.DataFrame(data, columns=("Card Name", "Quantity", "Date"), index=[0])

            cards_df= pd.concat([cards_df, card_data], ignore_index=True)
        except NoSuchElementException:
            pass

    #scraping of deck done
    #display progress
    print(f"Deck {i} of {length}")
    
    #saving checkpoints
    if i % 50 == 0:
        cards_df.to_csv('cards_df.csv')
        save_state=[]
        
        for link in save_state_links[i+1:]:
            save_state.append(link)
            
        save_state_df = pd.DataFrame(save_state)
        save_state_df.to_csv('save_state.csv', index=False)
        print('Checkpoint saved')
        
    i+=1

Jul 30, 2022
Deck 1 of 10216
Jul 30, 2022
Deck 2 of 10216
Jul 30, 2022
Deck 3 of 10216
Jul 30, 2022
Deck 4 of 10216
Jul 30, 2022
Deck 5 of 10216
Jul 30, 2022
Deck 6 of 10216
Jul 30, 2022
Deck 7 of 10216
Jul 30, 2022
Deck 8 of 10216
Jul 30, 2022
Deck 9 of 10216
Jul 30, 2022
Deck 10 of 10216
Jul 30, 2022
Deck 11 of 10216
Jul 30, 2022
Deck 12 of 10216
Jul 30, 2022
Deck 13 of 10216
Jul 30, 2022
Deck 14 of 10216
Jul 30, 2022
Deck 15 of 10216
Jul 30, 2022
Deck 16 of 10216
Jul 30, 2022
Deck 17 of 10216
Jul 30, 2022
Deck 18 of 10216
Jul 30, 2022
Deck 19 of 10216
Jul 30, 2022
Deck 20 of 10216
Jul 30, 2022
Deck 21 of 10216
Jul 30, 2022
Deck 22 of 10216
Jul 30, 2022
Deck 23 of 10216
Jul 30, 2022
Deck 24 of 10216
Jul 30, 2022
Deck 25 of 10216
Jul 30, 2022
Deck 26 of 10216
Jul 30, 2022
Deck 27 of 10216
Jul 30, 2022
Deck 28 of 10216
Jul 30, 2022
Deck 29 of 10216
Jul 30, 2022
Deck 30 of 10216
Jul 30, 2022
Deck 31 of 10216
Jul 30, 2022
Deck 32 of 10216
Jul 30, 2022
Deck 33 of 10216
Jul 30, 2022
Deck 3

Deck 266 of 10216
Jul 29, 2022
Deck 267 of 10216
Jul 29, 2022
Deck 268 of 10216
Jul 29, 2022
Deck 269 of 10216
Jul 29, 2022
Deck 270 of 10216
Jul 29, 2022
Deck 271 of 10216
Jul 29, 2022
Deck 272 of 10216
Jul 29, 2022
Deck 273 of 10216
Jul 29, 2022
Deck 274 of 10216
Jul 29, 2022
Deck 275 of 10216
Jul 29, 2022
Deck 276 of 10216
Jul 28, 2022
Deck 277 of 10216
Jul 28, 2022
Deck 278 of 10216
Jul 28, 2022
Deck 279 of 10216
Jul 28, 2022
Deck 280 of 10216
Jul 28, 2022
Deck 281 of 10216
Jul 28, 2022
Deck 282 of 10216
Jul 28, 2022
Deck 283 of 10216
Jul 28, 2022
Deck 284 of 10216
Jul 28, 2022
Deck 285 of 10216
Jul 28, 2022
Deck 286 of 10216
Jul 28, 2022
Deck 287 of 10216
Jul 28, 2022
Deck 288 of 10216
Jul 28, 2022
Deck 289 of 10216
Jul 28, 2022
Deck 290 of 10216
Jul 28, 2022
Deck 291 of 10216
Jul 28, 2022
Deck 292 of 10216
Jul 28, 2022
Deck 293 of 10216
Jul 28, 2022
Deck 294 of 10216
Jul 28, 2022
Deck 295 of 10216
Jul 28, 2022
Deck 296 of 10216
Jul 27, 2022
Deck 297 of 10216
Jul 27, 2022
Deck 298

Jul 22, 2022
Deck 528 of 10216
Jul 22, 2022
Deck 529 of 10216
Jul 22, 2022
Deck 530 of 10216
Jul 22, 2022
Deck 531 of 10216
Jul 22, 2022
Deck 532 of 10216
Jul 22, 2022
Deck 533 of 10216
Jul 22, 2022
Deck 534 of 10216
Jul 22, 2022
Deck 535 of 10216
Jul 22, 2022
Deck 536 of 10216
Jul 22, 2022
Deck 537 of 10216
Jul 22, 2022
Deck 538 of 10216
Jul 22, 2022
Deck 539 of 10216
Jul 22, 2022
Deck 540 of 10216
Jul 22, 2022
Deck 541 of 10216
Jul 22, 2022
Deck 542 of 10216
Jul 22, 2022
Deck 543 of 10216
Jul 22, 2022
Deck 544 of 10216
Jul 22, 2022
Deck 545 of 10216
Jul 22, 2022
Deck 546 of 10216
Jul 22, 2022
Deck 547 of 10216
Jul 22, 2022
Deck 548 of 10216
Jul 22, 2022
Deck 549 of 10216
Jul 22, 2022
Deck 550 of 10216
Checkpoint saved
Jul 22, 2022
Deck 551 of 10216
Jul 22, 2022
Deck 552 of 10216
Jul 22, 2022
Deck 553 of 10216
Jul 22, 2022
Deck 554 of 10216
Jul 22, 2022
Deck 555 of 10216
Jul 22, 2022
Deck 556 of 10216
Jul 22, 2022
Deck 557 of 10216
Jul 22, 2022
Deck 558 of 10216
Jul 22, 2022
Deck 559 

Jul 15, 2022
Deck 790 of 10216
Jul 15, 2022
Deck 791 of 10216
Jul 15, 2022
Deck 792 of 10216
Jul 15, 2022
Deck 793 of 10216
Jul 15, 2022
Deck 794 of 10216
Jul 15, 2022
Deck 795 of 10216
Jul 15, 2022
Deck 796 of 10216
Jul 15, 2022
Deck 797 of 10216
Jul 15, 2022
Deck 798 of 10216
Jul 15, 2022
Deck 799 of 10216
Jul 15, 2022
Deck 800 of 10216
Checkpoint saved
Jul 15, 2022
Deck 801 of 10216
Jul 15, 2022
Deck 802 of 10216
Jul 15, 2022
Deck 803 of 10216
Jul 15, 2022
Deck 804 of 10216
Jul 15, 2022
Deck 805 of 10216
Jul 15, 2022
Deck 806 of 10216
Jul 15, 2022
Deck 807 of 10216
Jul 15, 2022
Deck 808 of 10216
Jul 15, 2022
Deck 809 of 10216
Jul 15, 2022
Deck 810 of 10216
Jul 15, 2022
Deck 811 of 10216
Jul 15, 2022
Deck 812 of 10216
Jul 15, 2022
Deck 813 of 10216
Jul 14, 2022
Deck 814 of 10216
Jul 14, 2022
Deck 815 of 10216
Jul 14, 2022
Deck 816 of 10216
Jul 14, 2022
Deck 817 of 10216
Jul 14, 2022
Deck 818 of 10216
Jul 14, 2022
Deck 819 of 10216
Jul 14, 2022
Deck 820 of 10216
Jul 14, 2022
Deck 821 

Jul 10, 2022
Deck 1050 of 10216
Checkpoint saved
Jul 10, 2022
Deck 1051 of 10216
Jul 10, 2022
Deck 1052 of 10216
Jul 10, 2022
Deck 1053 of 10216
Jul 10, 2022
Deck 1054 of 10216
Jul 10, 2022
Deck 1055 of 10216
Jul 10, 2022
Deck 1056 of 10216
Jul 10, 2022
Deck 1057 of 10216
Jul 10, 2022
Deck 1058 of 10216
Jul 10, 2022
Deck 1059 of 10216
Jul 10, 2022
Deck 1060 of 10216
Jul 10, 2022
Deck 1061 of 10216
Jul 10, 2022
Deck 1062 of 10216
Jul 10, 2022
Deck 1063 of 10216
Jul 10, 2022
Deck 1064 of 10216
Jul 10, 2022
Deck 1065 of 10216
Jul 10, 2022
Deck 1066 of 10216
Jul 10, 2022
Deck 1067 of 10216
Jul 10, 2022
Deck 1068 of 10216
Jul 10, 2022
Deck 1069 of 10216
Jul 10, 2022
Deck 1070 of 10216
Jul 10, 2022
Deck 1071 of 10216
Jul 10, 2022
Deck 1072 of 10216
Jul 10, 2022
Deck 1073 of 10216
Jul 10, 2022
Deck 1074 of 10216
Jul 10, 2022
Deck 1075 of 10216
Jul 10, 2022
Deck 1076 of 10216
Jul 10, 2022
Deck 1077 of 10216
Jul 10, 2022
Deck 1078 of 10216
Jul 10, 2022
Deck 1079 of 10216
Jul 10, 2022
Deck 1080 

...

Feb 4, 2022
Deck 9005 of 10216
Feb 4, 2022
Deck 9006 of 10216
Feb 4, 2022
Deck 9007 of 10216
Feb 4, 2022
Deck 9008 of 10216
Feb 4, 2022
Deck 9009 of 10216
Feb 4, 2022
Deck 9010 of 10216
Feb 4, 2022
Deck 9011 of 10216
Feb 4, 2022
Deck 9012 of 10216
Feb 4, 2022
Deck 9013 of 10216
Feb 4, 2022
Deck 9014 of 10216
Feb 4, 2022
Deck 9015 of 10216
Feb 4, 2022
Deck 9016 of 10216
Feb 4, 2022
Deck 9017 of 10216
Feb 4, 2022
Deck 9018 of 10216
Feb 4, 2022
Deck 9019 of 10216
Feb 4, 2022
Deck 9020 of 10216
Feb 4, 2022
Deck 9021 of 10216
Feb 4, 2022
Deck 9022 of 10216
Feb 4, 2022
Deck 9023 of 10216
Feb 4, 2022
Deck 9024 of 10216
Feb 4, 2022
Deck 9025 of 10216
Feb 4, 2022
Deck 9026 of 10216
Feb 3, 2022
Deck 9027 of 10216
Feb 3, 2022
Deck 9028 of 10216
Feb 3, 2022
Deck 9029 of 10216
Feb 3, 2022
Deck 9030 of 10216
Feb 3, 2022
Deck 9031 of 10216
Feb 3, 2022
Deck 9032 of 10216
Feb 3, 2022
Deck 9033 of 10216
Feb 3, 2022
Deck 9034 of 10216
Feb 2, 2022
Deck 9035 of 10216
Feb 2, 2022
Deck 9036 of 10216
Feb 2, 2

Deck 9262 of 10216
Jan 28, 2022
Deck 9263 of 10216
Jan 28, 2022
Deck 9264 of 10216
Jan 28, 2022
Deck 9265 of 10216
Jan 28, 2022
Deck 9266 of 10216
Jan 28, 2022
Deck 9267 of 10216
Jan 28, 2022
Deck 9268 of 10216
Jan 28, 2022
Deck 9269 of 10216
Jan 28, 2022
Deck 9270 of 10216
Jan 28, 2022
Deck 9271 of 10216
Jan 28, 2022
Deck 9272 of 10216
Jan 28, 2022
Deck 9273 of 10216
Jan 28, 2022
Deck 9274 of 10216
Jan 28, 2022
Deck 9275 of 10216
Jan 28, 2022
Deck 9276 of 10216
Jan 28, 2022
Deck 9277 of 10216
Jan 28, 2022
Deck 9278 of 10216
Jan 28, 2022
Deck 9279 of 10216
Jan 28, 2022
Deck 9280 of 10216
Jan 28, 2022
Deck 9281 of 10216
Jan 28, 2022
Deck 9282 of 10216
Jan 28, 2022
Deck 9283 of 10216
Jan 28, 2022
Deck 9284 of 10216
Jan 28, 2022
Deck 9285 of 10216
Jan 28, 2022
Deck 9286 of 10216
Jan 28, 2022
Deck 9287 of 10216
Jan 28, 2022
Deck 9288 of 10216
Jan 28, 2022
Deck 9289 of 10216
Jan 28, 2022
Deck 9290 of 10216
Jan 28, 2022
Deck 9291 of 10216
Jan 28, 2022
Deck 9292 of 10216
Jan 28, 2022
Deck 929

Jan 21, 2022
Deck 9516 of 10216
Jan 21, 2022
Deck 9517 of 10216
Jan 21, 2022
Deck 9518 of 10216
Jan 21, 2022
Deck 9519 of 10216
Jan 21, 2022
Deck 9520 of 10216
Jan 21, 2022
Deck 9521 of 10216
Jan 21, 2022
Deck 9522 of 10216
Jan 21, 2022
Deck 9523 of 10216
Jan 21, 2022
Deck 9524 of 10216
Jan 21, 2022
Deck 9525 of 10216
Jan 21, 2022
Deck 9526 of 10216
Jan 21, 2022
Deck 9527 of 10216
Jan 21, 2022
Deck 9528 of 10216
Jan 21, 2022
Deck 9529 of 10216
Jan 21, 2022
Deck 9530 of 10216
Jan 21, 2022
Deck 9531 of 10216
Jan 21, 2022
Deck 9532 of 10216
Jan 21, 2022
Deck 9533 of 10216
Jan 21, 2022
Deck 9534 of 10216
Jan 21, 2022
Deck 9535 of 10216
Jan 21, 2022
Deck 9536 of 10216
Jan 21, 2022
Deck 9537 of 10216
Jan 21, 2022
Deck 9538 of 10216
Jan 21, 2022
Deck 9539 of 10216
Jan 21, 2022
Deck 9540 of 10216
Jan 21, 2022
Deck 9541 of 10216
Jan 21, 2022
Deck 9542 of 10216
Jan 21, 2022
Deck 9543 of 10216
Jan 21, 2022
Deck 9544 of 10216
Jan 21, 2022
Deck 9545 of 10216
Jan 21, 2022
Deck 9546 of 10216
Jan 21, 

Deck 9769 of 10216
Jan 16, 2022
Deck 9770 of 10216
Jan 16, 2022
Deck 9771 of 10216
Jan 16, 2022
Deck 9772 of 10216
Jan 16, 2022
Deck 9773 of 10216
Jan 16, 2022
Deck 9774 of 10216
Jan 16, 2022
Deck 9775 of 10216
Jan 16, 2022
Deck 9776 of 10216
Jan 16, 2022
Deck 9777 of 10216
Jan 16, 2022
Deck 9778 of 10216
Jan 16, 2022
Deck 9779 of 10216
Jan 16, 2022
Deck 9780 of 10216
Jan 16, 2022
Deck 9781 of 10216
Jan 16, 2022
Deck 9782 of 10216
Jan 16, 2022
Deck 9783 of 10216
Jan 16, 2022
Deck 9784 of 10216
Jan 15, 2022
Deck 9785 of 10216
Jan 15, 2022
Deck 9786 of 10216
Jan 15, 2022
Deck 9787 of 10216
Jan 15, 2022
Deck 9788 of 10216
Jan 15, 2022
Deck 9789 of 10216
Jan 15, 2022
Deck 9790 of 10216
Jan 15, 2022
Deck 9791 of 10216
Jan 15, 2022
Deck 9792 of 10216
Jan 15, 2022
Deck 9793 of 10216
Jan 15, 2022
Deck 9794 of 10216
Jan 15, 2022
Deck 9795 of 10216
Jan 15, 2022
Deck 9796 of 10216
Jan 15, 2022
Deck 9797 of 10216
Jan 15, 2022
Deck 9798 of 10216
Jan 15, 2022
Deck 9799 of 10216
Jan 15, 2022
Deck 980

Deck 10023 of 10216
Jan 8, 2022
Deck 10024 of 10216
Jan 8, 2022
Deck 10025 of 10216
Jan 8, 2022
Deck 10026 of 10216
Jan 8, 2022
Deck 10027 of 10216
Jan 8, 2022
Deck 10028 of 10216
Jan 8, 2022
Deck 10029 of 10216
Jan 8, 2022
Deck 10030 of 10216
Jan 8, 2022
Deck 10031 of 10216
Jan 8, 2022
Deck 10032 of 10216
Jan 8, 2022
Deck 10033 of 10216
Jan 8, 2022
Deck 10034 of 10216
Jan 8, 2022
Deck 10035 of 10216
Jan 8, 2022
Deck 10036 of 10216
Jan 8, 2022
Deck 10037 of 10216
Jan 8, 2022
Deck 10038 of 10216
Jan 8, 2022
Deck 10039 of 10216
Jan 8, 2022
Deck 10040 of 10216
Jan 8, 2022
Deck 10041 of 10216
Jan 8, 2022
Deck 10042 of 10216
Jan 8, 2022
Deck 10043 of 10216
Jan 8, 2022
Deck 10044 of 10216
Jan 8, 2022
Deck 10045 of 10216
Jan 8, 2022
Deck 10046 of 10216
Jan 8, 2022
Deck 10047 of 10216
Jan 8, 2022
Deck 10048 of 10216
Jan 8, 2022
Deck 10049 of 10216
Jan 8, 2022
Deck 10050 of 10216
Checkpoint saved
Jan 8, 2022
Deck 10051 of 10216
Jan 8, 2022
Deck 10052 of 10216
Jan 7, 2022
Deck 10053 of 10216
Jan

In [43]:
cards_df.to_csv('cards_df.csv')